# Anime Recommendation Database 2020

https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020?resource=download

In [50]:
#importamos librerias 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import warnings
warnings.filterwarnings("ignore")
import random
from scipy.sparse import csr_matrix

In [2]:
path = 'C:/Users/guill/Documents/DataScience/Data_Science_ICARO_UNC/Sprint3/Datasets/archive'

## Dataset

In [3]:
df_animelist = pd.read_csv(f'{path}/rating_complete.csv') 
#df_animelist = df_animelist.sample(10000)

In [4]:
df_animelist

,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9
...,...,...,...
57633273,353404,502,8
57633274,353404,987,4
57633275,353404,225,8
57633276,353404,243,7


In [6]:
df_animelist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57633278 entries, 0 to 57633277
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 1.3 GB


In [7]:
print(f'El dataset contiene {df_animelist.shape[0]} observaciones y {df_animelist.shape[1]} variables (features)')

El dataset contiene 57633278 observaciones y 3 variables (features)


**Features**

* user_id: non identifiable randomly generated user id.
* anime_id: MyAnemlist ID of the anime. (e.g. 1).
* score: score between 1 to 10 given by the user. 0 if the user didn't assign a score. (e.g. 10)
* watching_status: state ID from this anime in the anime list of this user. (e.g. 2)
* watched_episodes: numbers of episodes watched by the user. (e.g. 24

Tiene nulos?

In [33]:
df_animelist.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

Cuantos usuarios tenemos?

In [9]:
df_animelist.user_id.nunique()

310059

Tenemos 310059 usuarios

## Train y Test

57633278 es el total de filas, si dividimos en 70-30. El train seria 40343295 y test 17289983

In [20]:
random.seed(42)
Train = df_animelist.sample(40343295)

In [21]:
Train.head()

,user_id,anime_id,rating
30883805,189756,36538,8
52661552,322853,160,8
37655964,231234,384,9
33982108,208616,3784,9
27623961,170037,11617,7


In [22]:
Test = df_animelist[~df_animelist.isin(Train)].copy()
Test = Test.dropna()
Test

,user_id,anime_id,rating
0,0.0,430.0,9.0
3,0.0,570.0,7.0
8,0.0,1571.0,10.0
9,0.0,121.0,9.0
12,0.0,2913.0,6.0
...,...,...,...
57633268,353404.0,894.0,7.0
57633269,353404.0,813.0,9.0
57633270,353404.0,893.0,8.0
57633274,353404.0,987.0,4.0


In [23]:
Train.user_id.nunique()

307891

In [24]:
Test.user_id.nunique()

300704

Tenemos usuarios que aparezcan en el test y que no esten estén en el train?

In [25]:
Test[~Test.user_id.isin(Train.user_id.unique())].user_id.nunique()

2168

Ahi tendremos que aplicar **cold start**

## Matriz de interacciones

In [43]:
Train_ = Train.sample(10000)

In [44]:
Train_

,user_id,anime_id,rating
46464975,284962,12413,9
26729269,164482,896,6
27687616,170425,31737,5
30475885,187438,28891,9
32327724,198541,26,8
...,...,...,...
32893848,202028,11887,7
39068933,239969,341,9
55099673,337698,7088,10
6907793,42429,15809,9


In [45]:
interactions_matrix = Train_.pivot(index="user_id", columns="anime_id", values="rating")
interactions_matrix.head()

anime_id,1,5,6,7,16,18,19,20,22,24,...,41619,41926,42057,42603,42668,42723,43467,43555,43779,47398
user_id,,,,,,,,,,,,,,,,,,,,,
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#completamos NaN con 0
interactions_matrix = interactions_matrix.fillna(0)
interactions_matrix.head()

anime_id,1,5,6,7,16,18,19,20,22,24,...,41619,41926,42057,42603,42668,42723,43467,43555,43779,47398
user_id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
interactions_matrix.shape

(9592, 3398)

transformamos la matriz a csr matrix (compressed sparse row)

In [51]:
interactions_matrix_csr = csr_matrix(interactions_matrix.values)
interactions_matrix_csr

<9592x3398 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [52]:
user_ids = list(interactions_matrix.index)
user_dict = {}
counter = 0 
for i in user_ids:
    user_dict[i] = counter
    counter += 1

In [53]:
user_dict

{3: 0,
 41: 1,
 51: 2,
 85: 3,
 147: 4,
 160: 5,
 164: 6,
 204: 7,
 320: 8,
 418: 9,
 446: 10,
 461: 11,
 473: 12,
 554: 13,
 577: 14,
 598: 15,
 601: 16,
 644: 17,
 647: 18,
 669: 19,
 678: 20,
 732: 21,
 787: 22,
 857: 23,
 883: 24,
 890: 25,
 912: 26,
 960: 27,
 1042: 28,
 1177: 29,
 1189: 30,
 1192: 31,
 1209: 32,
 1239: 33,
 1296: 34,
 1297: 35,
 1303: 36,
 1358: 37,
 1397: 38,
 1473: 39,
 1639: 40,
 1670: 41,
 1690: 42,
 1695: 43,
 1697: 44,
 1775: 45,
 1879: 46,
 1938: 47,
 2021: 48,
 2065: 49,
 2081: 50,
 2151: 51,
 2155: 52,
 2220: 53,
 2283: 54,
 2317: 55,
 2346: 56,
 2347: 57,
 2355: 58,
 2411: 59,
 2425: 60,
 2454: 61,
 2576: 62,
 2596: 63,
 2605: 64,
 2691: 65,
 2710: 66,
 2780: 67,
 2787: 68,
 2814: 69,
 2822: 70,
 2855: 71,
 2885: 72,
 2892: 73,
 2938: 74,
 2970: 75,
 2976: 76,
 2982: 77,
 3005: 78,
 3053: 79,
 3081: 80,
 3139: 81,
 3185: 82,
 3187: 83,
 3290: 84,
 3337: 85,
 3363: 86,
 3397: 87,
 3428: 88,
 3436: 89,
 3440: 90,
 3459: 91,
 3502: 92,
 3516: 93,
 3532: 94

## Modelo